# RDD & pandas 방식 차감 성능 테스트  
- rdd cache 이용한 차감 전체 차감 속도 측정(stream server 안에서)  
- pandas 이용한 로컬 차감 속도 측정  


In [14]:
# 필요 라이브러리 임포트  
import socket
import sys
import os
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession
from os.path import abspath
import findspark
import time 
import pandas as pd

In [3]:
# # !pip install jupyter-resource-usage jupyterlab-system-monitor jupyterlab-topbar-extension
# !pip install jupyterlab-topbar jupyterlab-topbar-text jupyterlab-topbar-extension

In [15]:
# 환경변수 정의  
scale = 1000 # 1000 만 건 수준
PRJ_ROOT = '/user/root'
APP_NAME = 'RDD-Pandas'
DB_NAME = 'inven'

# 데이터의 파일 포맷 및 파일명  
tbl_setop_name = 'inven/table-set-6m-20-1000'
file_format = 'parquet'

In [16]:
# 스파크 생성 
def spark_creation():
    spark = SparkSession.builder.master('yarn').appName(APP_NAME)\
    .config('spark.driver.cores', '1').config('spark.driver.memory', '7g')\
    .config('spark.num.executors', '3')\
    .config('spark.executor.cores', '1').config('spark.executor.memory', '7g')\
    .config('spark.jars', '/hive-bin/lib/mysql-connector-java-5.1.49-bin.jar')\
    .config('spark.driver.extraClassPath', '/hive-bin/lib/mysql-connector-java-5.1.49-bin.jar').getOrCreate()
    sc = spark.sparkContext
    sc
    return spark

In [17]:
%%time
spark = spark_creation()
spark

CPU times: user 15.6 ms, sys: 7.51 ms, total: 23.1 ms
Wall time: 30.2 s


/usr/local/lib/python3.6/dist-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


### 데이터 적재  

In [7]:
%%time
## 인벤 기준 정보 조회  
spark.read.format(file_format).load(tbl_setop_name).createOrReplaceTempView('setop_view')
spark.catalog.cacheTable("setop_view")
spark.catalog.isCached('setop_view')

CPU times: user 3.16 ms, sys: 1.74 ms, total: 4.89 ms
Wall time: 4.33 s


True

In [11]:
%%time
# toPandas : 80 초(2cols),   
df = spark.sql("select * from setop_view ").rdd.toDF().toPandas()

Py4JJavaError: An error occurred while calling o124.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 6.0 failed 4 times, most recent failure: Lost task 0.3 in stage 6.0 (TID 13) (dn01 executor 4): ExecutorLostFailure (executor 4 exited caused by one of the running tasks) Reason: Executor Process Lost
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2303)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2252)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2251)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2251)
	at org.apache.spark.scheduler.DA

In [12]:
df.shape

NameError: name 'df' is not defined

In [10]:
del df

In [13]:
spark.stop()